In [23]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import keras_tuner as kt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from keras import regularizers
from keras.layers import Dropout, BatchNormalization
from keras_tuner.tuners import RandomSearch
from PIL import Image
from ast import literal_eval
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report

**Carga datos Train-Test-Valid**



In [2]:
dataset = pd.read_csv("data/birds.csv")
train = dataset.loc[dataset['data set'] == "train"].head(1100).reset_index()
test = dataset.loc[dataset['data set'] == "test"].reset_index()
valid = dataset.loc[dataset['data set'] == "valid"].reset_index()

In [3]:
train = train.drop(columns=['index'])
test = test.drop(columns=['index'])
valid = valid.drop(columns=['index'])

In [4]:
test.tail()

,class id,filepaths,labels,data set,scientific name
2620,524.0,test/BLACK BREASTED PUFFBIRD/3.jpg,BLACK BREASTED PUFFBIRD,test,NOTHARCHUS PECTORALIS
2621,524.0,test/BLACK BREASTED PUFFBIRD/4.jpg,BLACK BREASTED PUFFBIRD,test,NOTHARCHUS PECTORALIS
2622,524.0,test/BLACK BREASTED PUFFBIRD/1.jpg,BLACK BREASTED PUFFBIRD,test,NOTHARCHUS PECTORALIS
2623,524.0,test/BLACK BREASTED PUFFBIRD/2.jpg,BLACK BREASTED PUFFBIRD,test,NOTHARCHUS PECTORALIS
2624,524.0,test/BLACK BREASTED PUFFBIRD/5.jpg,BLACK BREASTED PUFFBIRD,test,NOTHARCHUS PECTORALIS


In [5]:
train["labels"].value_counts()

ABBOTTS BOOBY                 180
ABYSSINIAN GROUND HORNBILL    179
ABBOTTS BABBLER               163
AFRICAN EMERALD CUCKOO        154
AFRICAN OYSTER CATCHER        152
AFRICAN FIREFINCH             137
AFRICAN CROWNED CRANE         135
Name: labels, dtype: int64

Los datos no están muy balanceados, ya que hay especies con más fotos que otras.

**One Hot Encoding de las variables objetivo**

In [6]:
labels = ["ABBOTTS BOOBY", "ABYSSINIAN GROUND HORNBILL", "ABBOTTS BABBLER", "AFRICAN EMERALD CUCKOO", "AFRICAN OYSTER CATCHER", "AFRICAN FIREFINCH", "AFRICAN CROWNED CRANE"]

In [7]:
X_train = []
Y_train = []
for i in train.index:
    label = train["labels"][i]
    path = "data/" + train["filepaths"][i]
    image = Image.open(path)
    array = np.array(image)
    X_train.append(array)
    Y_train.append(label)
X_train = np.array(X_train)/255
Y_train = np.array(Y_train)

In [ ]:
X_test = []
Y_test = []
for i in range(35):
    label = test["labels"][i]
    path = "data/" + test["filepaths"][i]
    image = Image.open(path)
    array = np.array(image)
    X_test.append(array)
    Y_test.append(label)
X_test = np.array(X_test)/255
Y_test = np.array(Y_test)

In [ ]:
X_valid = []
Y_valid = []
for i in range(35):
    label = valid["labels"][i]
    path = "data/" + valid["filepaths"][i]
    image = Image.open(path)
    array = np.array(image)
    X_valid.append(array)
    Y_valid.append(label)
X_valid = np.array(X_valid)/255
Y_valid = np.array(Y_valid)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_valid.shape

In [ ]:
enc = preprocessing.OrdinalEncoder(categories=[labels])
Y_train = enc.fit_transform(Y_train.reshape(-1, 1)).ravel()
Y_test = enc.fit_transform(Y_test.reshape(-1, 1)).ravel()
Y_valid = enc.fit_transform(Y_valid.reshape(-1, 1)).ravel()

**Creación del modelo CNN**

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10, # Degree range for random rotations.
    zoom_range=0.1, # Range for random zoom
    width_shift_range=0.1, # fraction of total width (traslation)
    height_shift_range=0.1, # fraction of total height (traslation)
    horizontal_flip=False, # Randomly flip inputs horizontally.
    vertical_flip=False # Randomly flip inputs vertically.
)
datagen.fit(X_train)

In [ ]:
model = Sequential()
input_shape=(224, 224, 3)

# Convolution layer 1
model.add(Conv2D(filters=120, kernel_size=(3,3), strides=(1,1),padding='same',activation='relu',input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2,2)))
# Convolution layer 2
model.add(Conv2D(filters=100, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
# Convolution layer 3
model.add(Conv2D(filters=80, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
# Convolution layer 4
model.add(Conv2D(filters=60, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# Flatten Layers
model.add(Flatten())

# Dense layers
model.add(Dense(units=1000,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(units=1000,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1000,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=len(labels) ,activation='softmax'))

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=64),
    epochs=20,
    validation_data=(X_valid, Y_valid),
    verbose=2,
    steps_per_epoch=X_train.shape[0] // 64,
    callbacks=[learning_rate_reduction]
)

**Evaluación del modelo**

In [ ]:
loss, accuracy = model.evaluate(X_train, Y_train)
print("Final training loss: {:.8f}".format(loss))
print("Final training accuracy: {:.4%}".format(accuracy))

In [ ]:
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

El modelo resultó ser bastante malo, por lo que buscaremos los mejores hiperparámetros posibles.

In [ ]:
def build_model(hp):
    initializer = initializers.HeNormal(seed=20)
    model = Sequential()    
    regu = hp.Choice('l1_reg', [0.001, 0.0001])
    input_shape=(200, 200, 3)
    
    #Conv layer loop
    for i in range(4):
        nodes = hp.Choice(f'nodes_{i+1}', [32, 64, 128, 256, 512])
        model.add(Conv2D(nodes, (3, 3), activation='relu', input_shape=input_shape, kernel_initializer=initializer, kernel_regularizer=regularizers.l1(regu)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(BatchNormalization(momentum=hp.Float('momentum', min_value=0.8, max_value=0.9, step=0.05), epsilon=0.001))
    model.add(Flatten())
    
    #Fully Connected layers
    for i in range(4):
        units = hp.Choice(f'units_{i+1}', [32, 64, 128, 256, 512])
        model.add(Dense(units, activation='relu',kernel_initializer=initializer,kernel_regularizer=regularizers.l1(regu)))
        model.add(BatchNormalization(momentum=hp.Float('momentum', min_value=0.8, max_value=0.95, step=0.05), epsilon=0.001))
     
    model.add(Dense(5, activation='softmax', kernel_initializer=initializer))

    opt = Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0001]))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
#Using a bayesian optimizer
tuner = keras_tuner.BayesianOptimization(
    build_model,
    objective=keras_tuner.Objective("val_loss", direction="min"),
    max_trials=70, 
    num_initial_points=35,  
    overwrite=True  
)
tuner.search(train_main, epochs=5, validation_data=(valid_main))

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0]
print("Best Hyperparameters:")
for param, value in best_hp.values.items():
    print(f"{param}: {value}")

# Print the best model summary
best_model = tuner.get_best_models()[0]

In [ ]:
initializer = initializers.HeNormal(seed=20)
model1 = Sequential()
    
for i in range(4):        
    
    model1.add(Conv2D(best_hp.get(f'nodes_{i+1}'), (3, 3), activation='relu', kernel_initializer=initializer, kernel_regularizer=regularizers.l1(best_hp.get('l1_reg'))))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(BatchNormalization(momentum=best_hp.get(f'momentum'), epsilon=0.001))
    
model1.add(Flatten())
    
for i in range(4):
    model1.add(Dense(best_hp.get(f'units_{i+1}'),activation='relu',kernel_initializer=initializer,kernel_regularizer=regularizers.l1(best_hp.get('l1_reg'))))
    model1.add(BatchNormalization(momentum=best_hp.get(f'momentum'), epsilon=0.001))
        
model1.add(Dense(5, activation='softmax', kernel_initializer=initializer))

opt = Adam(learning_rate=best_hp.get('learning_rate'))
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
fin_hist = model1.fit(train_main, steps_per_epoch = len(train_main), epochs = 8, validation_data=valid_main)
# Evaluate the model
loss, accuracy = model1.evaluate(test_main)
print("Final training loss: {:.8f}".format(loss))
print("Final training accuracy: {:.4%}".format(accuracy))

In [ ]:
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()